In [ ]:
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta

In [17]:
import pandas as pd

df_prices = pd.read_csv('BelpexFilter.csv', delimiter=';')
df_prices.rename(columns={'Date':'timestamp'}, inplace=True)
df_prices.timestamp = pd.to_datetime(df_prices.timestamp)
df_prices.sort_values('timestamp', inplace=True)
df_prices.set_index('timestamp', inplace=True)
df_prices = df_prices.asfreq('H')
df_prices = df_prices.resample('15T').interpolate()

# df_prices.dynamic_price = df_prices.dynamic_price*1e-3 + 0.204*1e-3
# df_prices['flat_price'] = np.mean(df_prices.dynamic_price)

df_prices.head()

,energy_price
timestamp,
2017-01-01 00:00:00,42.000
2017-01-01 00:15:00,43.985
2017-01-01 00:30:00,45.970
2017-01-01 00:45:00,47.955
2017-01-01 01:00:00,49.940


In [ ]:
# df = pd.read_csv('Productie en verbruik info Core.csv', delimiter=';')
# df.head()

In [ ]:
# load1 = [0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0]
# load2 = [1,0,0,1,0,0,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,0,1,1]
# load1 = [val for val in load1 for _ in (0, 1, 2, 3)]
# load2 = [val for val in load2 for _ in (0, 1, 2, 3)]
# soc1 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# soc2 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# soc1 = [val for val in soc1 for _ in (0, 1, 2, 3)]
# soc2 = [val for val in soc2 for _ in (0, 1, 2, 3)]

# df_av = pd.DataFrame()
# df_av['timestamp'] = df.index
# df_av['availability_user1'] = load1*365
# df_av['availability_user2'] = load2*365
# df_av.set_index('timestamp', inplace=True)
# df_av.to_csv('availability_profiles.csv')

In [ ]:
pd.to_datetime(df.Datum.iloc[0] + ' ' + df.Tijd.iloc[0])
df.Datum = pd.to_datetime(df.Datum + ' ' + df.Tijd)
df.rename(columns={'Datum':'timestamp'}, inplace=True)
df.drop(['Tijd'], axis = 1, inplace = True)
df.set_index('timestamp', inplace=True)
df.head()

In [ ]:
# T = 24
# availability_user1 = np.array([0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])
# availability_user2 = np.array([0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])
from profiles import users

df.Beschikbaarheidsprofiel1.iloc[:96] = np.array(users[0].get('loadprof'))
df.Beschikbaarheidsprofiel2.iloc[:96] = np.array(users[1].get('loadprof'))

max_charge_rate = 22000/4
offtake_price = 0.5e-3


In [ ]:
plt.figure()
plt.plot(df.timestamp.iloc[:96], df.Beschikbaarheidsprofiel1.iloc[:96])
plt.plot(df.timestamp.iloc[:96], df.Beschikbaarheidsprofiel2.iloc[:96])
plt.legend(['Beschikbaarheid user 1','Beschikbaarheid user 2'], loc='upper right', bbox_to_anchor=(1.5, 1))


In [ ]:
# Dom laden: op elk tijdsstip t vraagt elke auto die wilt laden, max vermogen. Als boven cap: verschil met cap/#ladende auto's aftrekken van geladen vermogen.

In [ ]:
# Make Tz
status = 0
start = 0
stop = 0
Tz_user1 = []
for t in range(len(df.Beschikbaarheidsprofiel1.iloc[0:96])):
    if df.Beschikbaarheidsprofiel1.iloc[t] == 1 and status==0:
        start = t
        status = 1
    elif df.Beschikbaarheidsprofiel1.iloc[t] == 0 and status==1: #in deze t laadt hij niet meer, maar in de vorige wel nog -> stop = t
        stop = t
        status = 0
        Tz_user1.append((start,stop))

status = 0
start = 0
stop = 0
Tz_user2 = []
for t in range(len(df.Beschikbaarheidsprofiel2.iloc[0:96])):
    if df.Beschikbaarheidsprofiel2.iloc[t] == 1 and status==0:
        start = t
        status = 1
    elif df.Beschikbaarheidsprofiel2.iloc[t] == 0 and status==1: #in deze t laadt hij niet meer, maar in de vorige wel nog -> stop = t
        stop = t
        status = 0
        Tz_user2.append((start,stop))

Tz = [Tz_user1, Tz_user2]
print(Tz)

In [ ]:
max_charge_rates = [users[0].get('user')[0], users[1].get('user')[0]]

In [ ]:
print(users[1].get('demandprof'))
print(users[0].get('user'))

In [ ]:
demand = [[0,0], [0,0,0,0,0]]
for i in range(2):
    for y in range(len(demand[i])):
        demand[i][y] = (users[i].get('demandprof')[y][1]-users[i].get('demandprof')[y][0])*users[i].get('user')[1]*1000*4

print(demand)

In [ ]:
model = LpProblem(name='laadpaalstudie', sense=LpMinimize)
Z = [2,3] #number of times the car is at the chargingstation within simulation period (T), Z = len(Tz)
T = 24*4 #timesteps in hour resolution
cap = 20000
C = 2 #number of charging stations
# Tz = [[(1*4,6*4),(18*4,22*4)], [(3*4,4*4),(10*4,15*4),(19*4,22*4)]]
# demand = [[3000*4,5000*4],[1000*4,2000*4,6000*4]] #in Watt-kwartier
priority = [1,1] #According to users: least priority gets 1, next 2, next 3,... len(priority) = C


zcharge = LpVariable.dicts('zcharge', [(c,z) for c in range(C) for z in range(Z[c])], lowBound=0, upBound= max_charge_rate)#[max_charge_rates[c] for c in range(C)])
tcharge = LpVariable.dicts('tcharge', [(c,t) for c in range(C) for t in range(T)], lowBound=0, upBound=max_charge_rate)#[max_charge_rates[c] for c in range(C)])

obj = lpSum([(demand[c][z] - zcharge[(c,z)])*priority[c] for c in range(C) for z in range(Z[c])])
model += obj


for t in range(T):
    model += (lpSum(tcharge[(c,t)] for c in range(C)) + np.array(df['Gemeenschappelijk verbruik in kW'].iloc[t])*1000 - np.array(df['Productie in kW'].iloc[t])*1000 <= cap)
    
for c in range(C):
    for z in range(Z[c]):
        model += (zcharge[(c,z)] == lpSum([tcharge[(c,t)] for t in range(Tz[c][z][0],Tz[c][z][1])]))

        model += (zcharge[(c,z)] <= demand[c][z])



# Solve the problem
status = model.solve(use_mps=False)

print(f"status: {model.status}, {LpStatus[model.status]}")


print(f"objective: {model.objective.value()}")


for var in model.variables():
    print(f"{var.name}: {var.value()}")



In [ ]:
# zcharge_arr = np.zeros([(c,z) for c in range(C) for z in range(Z[c])])
tcharge_arr = np.zeros((C,T))
for c in range(C):
    for t in range(T):
        tcharge_arr[(c,t)] = tcharge[(c,t)].value()
    # for z in range(Z[c]):
    #     zcharge_arr[(c,z)] = zcharge[(c,z)].value()

In [ ]:
print(T)

In [ ]:
print(np.array(df.Beschikbaarheidsprofiel1.iloc[:96]))

In [ ]:
plt.figure()
plt.plot(np.array(df['Gemeenschappelijk verbruik in kW'].iloc[0:T])*1000)
plt.plot(np.array(df['Productie in kW'].iloc[0:T])*1000)
plt.plot(tcharge_arr[0])
# plt.plot(np.array(df.Beschikbaarheidsprofiel1.iloc[:96])*cap)
plt.plot(tcharge_arr[1])
# plt.plot(np.array(df.Beschikbaarheidsprofiel2.iloc[:96])*cap)
plt.plot(tcharge_arr[0] + tcharge_arr[1] + np.array(df['Gemeenschappelijk verbruik in kW'].iloc[0:T])*1000 - np.array(df['Productie in kW'].iloc[0:T])*1000)
plt.legend(['Gemeenschappelijk verbruik', 'Productie', 'Laadprofiel 1', 'Laadprofiel 2', 'Afname net'], loc='upper right', bbox_to_anchor=(1.5, 1))
#, 'Beschikbaarheid 1','Beschikbaarheid 2'


# for c in range(C):
#     plt.plot(tcharge_arr[c])
#     # plt.plot(zcharge_arr[c])
# print(tcharge_arr[0], tcharge_arr[1])
# # print(zcharge_arr[1])